<a href="https://colab.research.google.com/github/lmartin1999/baseball_testing/blob/main/bigb95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import random


data=pd.read_excel("fixed2.xlsx")
def play_hoh_competition(data, previous_hoh=None):
    eligible_players = data[data['Name'] != previous_hoh].reset_index(drop=True)
    total_weights = sum(eligible_players['Comp'])
    weights = [comp_rating/total_weights for comp_rating in eligible_players['Comp']]
    hoh = random.choices(eligible_players['Name'], weights)[0]
    return hoh

def nominate_with_checks(data, hoh):
    hoh_alliance = data.loc[data['Name'] == hoh, 'Alliance'].values[0]
    opposing_team = 2 if hoh_alliance == 1 else 1
    if opposing_team not in data['Alliance'].values:
        top_opposing_nominees = data.sort_values(by='Target', ascending=False).head(2)['Name'].tolist()
        return top_opposing_nominees
    opposing_team_members = data[data['Alliance'] == opposing_team].sort_values(by='Target', ascending=False)
    top_opposing_nominees = opposing_team_members.head(2)['Name'].tolist()
    if len(top_opposing_nominees) < 2:
        if 3 in data['Alliance'].values:
            deadman_nominee = data[data['Alliance'] == 3].sort_values(by='Target', ascending=False).head(1)['Name'].values[0]
            top_opposing_nominees.append(deadman_nominee)
        else:
            additional_nominee = data[~data['Name'].isin(top_opposing_nominees)].sort_values(by='Target', ascending=False).head(1)['Name'].values[0]
            top_opposing_nominees.append(additional_nominee)
    return top_opposing_nominees

def play_veto_competition_with_checks(data, hoh, nominees):
    players = [hoh] + nominees
    remaining_spots = 6 - len(players)
    if remaining_spots > 0:
        remaining_players = data[~data['Name'].isin(players)]['Name'].tolist()
        additional_players = random.sample(remaining_players, min(remaining_spots, len(remaining_players)))
        players += additional_players
    weights = [data[data['Name'] == player]['Comp'].values[0] for player in players]
    total_weight = sum(weights)
    normalized_weights = [weight/total_weight for weight in weights]
    veto_winner = random.choices(players, normalized_weights)[0]
    return veto_winner

def decide_veto_use(data, hoh, veto_winner, nominees):
    if veto_winner in nominees:
        return [player for player in nominees if player != veto_winner] + [hoh]
    hoh_alliance = data.loc[data['Name'] == hoh, 'Alliance'].values[0]
    opposing_team = 2 if hoh_alliance == 1 else 1
    opposing_team_count = len(data[data['Alliance'] == opposing_team])
    if len(data[data['Alliance'] == hoh_alliance]) > opposing_team_count:
        potential_backdoor_targets = data[(data['Alliance'] == hoh_alliance) & (data['Name'] != hoh)]
        if not potential_backdoor_targets.empty:
            sorted_targets = potential_backdoor_targets.sort_values(by=['Target', 'Comp'], ascending=[False, False])
            backdoor_target = sorted_targets.head(1)['Name'].values[0]
            if backdoor_target not in nominees:
                return [nominees[0], backdoor_target]
    return nominees

def vote_to_evict(data, nominees):
    votes = {}
    for nominee in nominees:
        votes[nominee] = 0
    for player in data['Name']:
        if player not in nominees:
            chosen_nominee = random.choice(nominees)
            votes[chosen_nominee] += 1
    sorted_votes = sorted(votes.items(), key=lambda x: x[1], reverse=True)
    evicted = sorted_votes[0][0]

    return evicted

def run_final_three_simulations(data, number_of_simulations=1000):
    final_three_players = {name: 0 for name in data['Name']}
    for _ in range(number_of_simulations):
        simulation_data = data.copy()
        previous_hoh = None
        while len(simulation_data) > 3:
            hoh = play_hoh_competition(simulation_data, previous_hoh)
            nominees = nominate_with_checks(simulation_data, hoh)
            veto_winner = play_veto_competition_with_checks(simulation_data, hoh, nominees)
            final_nominees = decide_veto_use(simulation_data, hoh, veto_winner, nominees)
            evicted_player = vote_to_evict(simulation_data, final_nominees)
            simulation_data = simulation_data[simulation_data['Name'] != evicted_player].reset_index(drop=True)
            previous_hoh = hoh
        for player in simulation_data['Name']:
            final_three_players[player] += 1
    for player, count in final_three_players.items():
        final_three_players[player] = (count / number_of_simulations) * 100
    return final_three_players

# ---------------------- ADD THE BELOW LINES HERE ---------------------- #

# This will run the simulation and store the results in final_three_results
final_three_results = run_final_three_simulations(data)

# This will print out the results
for player, percentage in final_three_results.items():
    print(f"{player}: {percentage:.2f}%")


America: 4.20%
Bleu: 5.90%
Bowie: 71.40%
Cam: 24.40%
Cirie: 1.50%
Cory: 17.40%
Felicia: 31.20%
Izzy: 6.10%
Jag: 2.10%
Jared: 1.10%
Matt: 18.90%
Mecole: 53.80%
Red: 62.00%
